In [22]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from numpy import *

In [23]:
#fp = open("/home/hirak/rapmapClustering/sailfish/build/quant_yeast.clust")

In [24]:
s = "1       2735nt, >comp2876_c0_seq2... at +/97.44%"
s.split()[2].lstrip('>').rstrip('.')

'comp2876_c0_seq2'

In [37]:
def readSailfishQuasi(fn, suffix=""):
    df = pd.read_table(fn, sep='\t', comment='#',
                     names=['Name',
                            'Length{}'.format(suffix),
                            'TPM{}'.format(suffix),
                            'NumReads{}'.format(suffix)])
    df.dropna(how='all', inplace=True)
    df.convert_objects(convert_numeric=True)
    df.set_index('Name', inplace=True)
    return df
sailfishFluxEM = readSailfishQuasi('/home/hirak/rapmapClustering/sailfish/build/data/human/quant_combined/quant.sf',"_sailfish")

In [44]:
sailfishFluxEM['TPM_sailfish']['comp4886_c0_seq1']

2.4053900000000001

In [61]:
def modifyGraph(fn,quantResult,w):
    fp = open(fn)
    fname = fn.split('.')[0]
    print fname 
    fp_mod = open(fname + '_'+'modified'+str(w)+'.net','w')
    for l in fp:
        tr_1,tr_2,ct = l[:-1].split('\t')
        if len(tr_2.split('_')) == 3:
            exp_1 = quantResult['TPM_sailfish'][tr_1]
            exp_2 = quantResult['TPM_sailfish'][tr_2]
            newWeight = w*int(ct) + (1-w) * np.reciprocal(np.abs(exp_1 - exp_2))
            fp_mod.write(tr_1 + '\t' + tr_2 + '\t' + str(newWeight) + '\n')
    return fp_mod

In [67]:
_ = modifyGraph('/home/hirak/rapmapClustering/sailfish/build/data/human/quant_combined.net',sailfishFluxEM,0.8)

/home/hirak/rapmapClustering/sailfish/build/data/human/quant_combined


In [42]:
s = "comp2876_c0_seq2"
print len(s.split('_'))

3


In [25]:
def readCDHitClust(fn, filtDict=None):
    tr_clust = {}
    fp = open(fn)
    cnum = None
    for l in fp:
        if l.startswith('>Cluster'):
            cnum = int(l.rstrip().split()[-1])
        else:
            e = l.split()[2].lstrip('>').rstrip('.')
            if not filtDict or e in filtDict:
                tr_clust[e] = cnum
    return tr_clust

In [26]:
def readMCLClust(fn):
    fp = open(fn)
    tr_clust = {}
    key = 1
    for cnum, line in enumerate(fp):
        same_cluster = line.rstrip().split('\t')
        for contig in same_cluster:
            tr_clust[contig] = cnum 
    return tr_clust

In [27]:
def readTrueLabels(fn):
    ft = open(fn)
    groundTruth_clust = {}
    gtClusterCount = {}
    for line in ft:
        tr_gn = line[:-1].split("\t")
        groundTruth_clust[tr_gn[0]] = tr_gn[1]
        if tr_gn[0] in gtClusterCount.keys():
            gtClusterCount[tr_gn[0]] += 1
        else:
            gtClusterCount[tr_gn[0]] = 1
    return groundTruth_clust

In [ ]:
def accuracyExpressed(groundTruth_clust, tr_clust):
    #count true postive for each pair of transcripts O(N^2)
    tp, fp, tn, fn = 0, 0, 0, 0
    for tr_1 in tr_cluststst.keys():
        for tr_2 in tr_clust.keys():
            if tr_1 != tr_2:
                if tr_1 not in groundTruth_clust or tr_2 not in groundTruth_clust:
                    continue
                ct = classType(groundTruth_clust[tr_1], groundTruth_clust[tr_2], tr_clust[tr_1], tr_clust[tr_2]) 
                if ct == Classification.TruePos:
                    tp += 1
                elif ct == Classification.TrueNeg:
                    tn += 1
                elif ct == Classification.FalsePos:
                    fp += 1
                elif ct == Classification.FalseNeg:
                    fn += 1
    return tp, fp, tn, fn

In [28]:
import itertools

class Classification:
    TruePos, FalsePos, TrueNeg, FalseNeg = range(4)

def classType(true1, true2, pred1, pred2):
    if true1 == true2:
        if pred1 == pred2:
            return Classification.TruePos
        else: # truely the same, predicted different
            return Classification.FalseNeg
    else: # truly different
        if pred1 == pred2: #predicted same
            return Classification.FalsePos
        else:
            return Classification.TrueNeg
        
    
def accuracyExpressed(groundTruth_clust, tr_clust):
    #count true postive for each pair of transcripts O(N^2)
    tp, fp, tn, fn = 0, 0, 0, 0
    for tr_1, tr_2 in itertools.combinations(tr_clust.keys(), 2):
        if tr_1 not in groundTruth_clust or tr_2 not in groundTruth_clust:
            continue
        ct = classType(groundTruth_clust[tr_1], groundTruth_clust[tr_2], tr_clust[tr_1], tr_clust[tr_2]) 
        if ct == Classification.TruePos:
            tp += 1
        elif ct == Classification.TrueNeg:
            tn += 1
        elif ct == Classification.FalsePos:
            fp += 1
        elif ct == Classification.FalseNeg:
            fn += 1
    return tp, fp, tn, fn

In [29]:
ft = "/mnt/scratch3/avi/clustering/human/truth/contigs2genes.disambiguous.txt"
groundTruth_clust = readTrueLabels(ft)

In [69]:
ft = "/home/hirak/rapmapClustering/sailfish/build/data/human/clusters/quant_combined_modified0.8.clust"
tr_clust = readMCLClust(ft)

In [ ]:
tr_clust_1 = readMCLClust("/home/hirak/rapmapClustering/sailfish/build/data/human/clusters/quant66.clust")
tr_clust_2 = readMCLClust("/home/hirak/rapmapClustering/sailfish/build/data/human/clusters/quant67.clust")
tr_clust_3 = readMCLClust("/home/hirak/rapmapClustering/sailfish/build/data/human/clusters/quant68.clust")
tr_clust_4 = readMCLClust("/home/hirak/rapmapClustering/sailfish/build/data/human/clusters/quant69.clust")
tr_clust_5 = readMCLClust("/home/hirak/rapmapClustering/sailfish/build/data/human/clusters/quant70.clust")
tr_clust_6 = readMCLClust("/home/hirak/rapmapClustering/sailfish/build/data/human/clusters/quant71.clust")

In [70]:
tp, fp, tn, fn = accuracyExpressed(groundTruth_clust, tr_clust)
print("tp : {}, fp : {}, tn : {}, fn : {}".format(tp, fp, tn, fn))

tp : 36725, fp : 2758, tn : 935706494, fn : 119239


In [81]:
print("prec: {}, recall: {}".format(tp / float(tp + fp), tp / float(tp + fn)))

prec: 0.969696117842, recall: 0.219092381467


In [ ]:
tp, fp, tn, fn = accuracyExpressed(groundTruth_clust, tr_clust_1)
print("tp : {}, fp : {}, tn : {}, fn : {}".format(tp, fp, tn, fn))

In [ ]:
print("prec: {}, recall: {}".format(tp / float(tp + fp), tp / float(tp + fn)))

In [ ]:
tp, fp, tn, fn = accuracyExpressed(groundTruth_clust, tr_clust_2)
print("tp : {}, fp : {}, tn : {}, fn : {}".format(tp, fp, tn, fn))

In [ ]:
print("prec: {}, recall: {}".format(tp / float(tp + fp), tp / float(tp + fn)))

In [ ]:
tp, fp, tn, fn = accuracyExpressed(groundTruth_clust, tr_clust_2)
print("tp : {}, fp : {}, tn : {}, fn : {}".format(tp, fp, tn, fn))

In [33]:
ft = "/mnt/scratch3/avi/clustering/human/cd_hit/0.95.clstr"
cd_clust = readCDHitClust(ft, filtDict=tr_clust)

In [35]:
tp, fp, tn, fn = accuracyExpressed(groundTruth_clust, cd_clust)
print("tp : {}, fp : {}, tn : {}, fn : {}".format(tp, fp, tn, fn))

tp : 21499, fp : 405, tn : 958295803, fn : 135946


In [36]:
print("prec: {}, recall: {}".format(tp / float(tp + fp), tp / float(tp + fn)))

prec: 0.981510226443, recall: 0.136549271174


In [ ]:
len(tr_clust.keys())

In [ ]:
"true positive: "+str(tp),"false positive: "+str(fp),"true negative: "+str(tn),"false negative: "+str(fn)

In [ ]:
pwd

In [ ]:
%matplotlib inline
plt.hist(tr_clust.values())

In [ ]:
%matplotlib inline
plt.hist(gtClusterCount.values())

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go